In [1]:
## BIBLIOTECAS

# Manipulação e Tratamento de dados ############################
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

import pyperclip
from tableone import TableOne

# Repositorio - Diretorio ############################
import shutil
import os

# Manipulação de Strings ############################
import re #regex
import unidecode #arrumar textos

#tempo
import time
from time import sleep

#ignorando Warning inuteis ############################
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Graficos ############################
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.dates as mdates

plt.style.use('dark_background')
plt.style.use('seaborn-v0_8-whitegrid')

import seaborn as sns
custom_palette = sns.color_palette("bright")
sns.palplot(custom_palette)
sns.set_palette("bright")

# Definindo o estilo de fonte e tema
import matplotlib.font_manager as fm #para alterar as fontes dos graficos
fonte = fm.FontProperties(family="Calibri", style="italic")
%matplotlib inline

# TESTE ESTATISTICOS ############################ ANOVA
import statistics
import math

from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.stattools import durbin_watson #teste de autocorrelação

ModuleNotFoundError: No module named 'tableone'

In [ ]:
inicio = time.time() #iniciando o cronometro

pasta = 'C:/Users/gabri/OneDrive/Documentos/Anestesia - CMA/Cultura de Segurança/1 - Dados Brutos'
df = pd.read_excel(pasta + '/' + "BANCO CULTURA.xlsx")
df.sample(5)

fim = time.time() #encerrando cronometro
print('Tempo para rodar a simulação:', round(fim - inicio, 3))

In [ ]:
len(df)

In [ ]:
print(df.columns)
display(df.sample(5))

In [ ]:
df = df.drop('Outro (especifique)', axis=1)
df = df.drop('Outro (especifique)2', axis=1)
df = df.drop('SEÇÃO I: Seus Comentários  Por favor, sinta-se à vontade para fazer quaisquer comentários sobre a segurança do paciente, erros ou  notificação de eventos em seu hospital.  ', axis=1)

df = df.drop(df[df['22. Meu supervisor/gerente ignora problemas recorrentes na segurança do paciente'] == 'Algumas vezes'].index)
df = df.reset_index(drop=True)

df['5. Em sua função, você tipicamente tem contato ou interação direta com os pacientes?  '].fillna('b. NÃO, tipicamente não tenho contato ou interação direta com os pacientes.', inplace = True)
df = df.drop('5. Em sua função, você tipicamente tem contato ou interação direta com os pacientes?  3', axis=1)
df = df.drop('5. Em sua função, você tipicamente tem contato ou interação direta com os pacientes?  32', axis=1)

coluna = '5. Em sua função, você tipicamente tem contato ou interação direta com os pacientes?  '
counts = df[coluna].value_counts()
percentages = df[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
res = pd.DataFrame({'counts': counts, 'percentage': percentages})
print(res)

df_backup = df.copy()

#df = df.drop('', axis=1)

In [ ]:
coluna = ' SEÇÃO H: 4. Qual é o seu cargo neste hospital? Marque UMA resposta que melhor descreva a sua função.'
print(df[coluna].value_counts(),'\n')

df["Cargo"] = np.nan
df["Cargo"][df[coluna] == 'Enfermeiro Supervisor/Coordenador/Gerente'] = 'Nursing Assistant'
df["Cargo"][df[coluna] == 'Enfermeiro(a) Assistencial'] = 'Nursing Assistant'
df["Cargo"][df[coluna] == 'Técnico de Enfermagem'] = 'Nursing Technician'
df["Cargo"][df[coluna] == 'Farmacêutico'] = 'Pharmacist'
df["Cargo"][df[coluna] == 'Ouvidor'] = 'Support Services'
df["Cargo"][df[coluna] == 'Assistente/Atendente/Secretária da Unidade'] = 'Support Services'
df["Cargo"][df[coluna] == 'Copeiro'] = 'Support Services'
df["Cargo"][df[coluna] == 'Administrativo (Recepções)'] = 'Administrative area'
df["Cargo"][df[coluna] == 'Psicologo'] = 'Psychologist'
df["Cargo"][df[coluna] == 'Médico plantonista terapia intensiva'] = 'Intensive Care Unit (ICU) Attending Physician'
df["Cargo"][df[coluna] == 'Radiologia)'] = 'Auxiliary Diagnostic and Therapeutic Services Physician'
df["Cargo"][df[coluna] == 'Fisioterapeuta'] = 'Physiotherapist'
df["Cargo"][df[coluna] == 'Nutricionista'] = 'Nutritionist'
df["Cargo"][df[coluna] == 'Técnico de exames (ex.: Eletrocardiograma Laboratório'] = 'Support Services'
df["Cargo"][df[coluna] == 'Auxiliar de Enfermagem'] = 'Nursing Technician'
df["Cargo"][df[coluna] == 'Médico staff cirúrgico'] = 'Surgical Staff Physician'
df["Cargo"][df[coluna] == 'Médico Gerente'] = 'Clinical Staff Physician'
df["Cargo"][df[coluna] == 'Médico staff clínico'] = 'Clinical Staff Physician'
df["Cargo"][df[coluna] == 'Concierge'] = 'Support Services'
df["Cargo"][df[coluna] == 'Farmacêutico Supervisor/Coordenador/Gerente'] = 'Pharmacist'
df["Cargo"][df[coluna] == 'Médico assistente -  clínico'] = 'Clinical Staff Physician'
df["Cargo"][df[coluna] == 'Assistente social'] = 'Support Services'
df["Cargo"][df[coluna] == 'Nutricionista Supervisor/Coordenador/Gerente'] = 'Nutritionist '
df["Cargo"][df[coluna] == 'Médico plantonista emergência'] = 'Clinical Staff Physician'
df["Cargo"][df[coluna] == 'Médico assistente - cirúrgico'] = 'Surgical Staff Physician'
df["Cargo"][df[coluna] == 'Médico SADT'] = 'Auxiliary Diagnostic and Therapeutic Services Physician'
df["Cargo"][df[coluna] == 'Fonoaudiólogo'] = 'Support Services'
df["Cargo"][df[coluna] == 'Terapeuta Ocupacional'] = 'Support Services'

df = df.drop(' SEÇÃO H: 4. Qual é o seu cargo neste hospital? Marque UMA resposta que melhor descreva a sua função.', axis=1)

print(df['Cargo'].value_counts(), '\n')

In [ ]:
sum(pd.isnull(df['Acreditação']))

In [ ]:
df['Acreditação'][df['Acreditação'] == 'Não possui'] = 0
df['Acreditação'][df['Acreditação'] == 0] = 0

df['Acreditação2'] = df['Acreditação']

df['Acreditação'][df['Acreditação'] == 'ONA 2'] = 'ONA'
df['Acreditação'][df['Acreditação'] == 'ONA 3'] = 'ONA'
df['Acreditação'][df['Acreditação'] == 'ONA 1'] = 'ONA'
df['Acreditação'][df['Acreditação'] == 'Qmentum'] = 'Qmentum'
df['Acreditação'][df['Acreditação'] == 'JCI'] = 'JCI'

df['Acreditação2'][df['Acreditação2'] == 'N1'] = 'ONA 1'
df['Acreditação'][df['Acreditação'] == 'N1'] = 'ONA'

print(df['Acreditação'].value_counts(),'\n')
print(df['Acreditação2'].value_counts(),'\n')

###################################################3#
df['acreditacao'] = 0
df['acreditacao'][df['Acreditação'] != 0] = 1
print(df['acreditacao'].value_counts())

In [ ]:
df.describe(include='all').T.head()

In [ ]:
for coluna in df.columns:
    print(coluna, '='*25)
    counts = df[coluna].value_counts()
    percentages = df[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

    # Usando o método 'agg' para juntar contagens e porcentagens em uma mesma DataFrame
    res = pd.DataFrame({'counts': counts, 'percentage': percentages})
    print(res)
    print('\n')

In [ ]:
df_alianca = df[(df['Unidade'] == 'Aliança') & (df['Ano'] == 2022)]

for coluna in df_alianca.columns:
    print(coluna, '='*25)
    counts = df_alianca[coluna].value_counts()
    percentages = df_alianca[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

    # Usando o método 'agg' para juntar contagens e porcentagens em uma mesma DataFrame
    res = pd.DataFrame({'counts': counts, 'percentage': percentages})
    print(res)
    print('\n')

In [ ]:
lista1 = []
lista2 = []

for coluna in df.columns:
    if 'Nunca + Raramente' in df[coluna].unique():
        lista1.append(coluna)
        #print('alteração 1:', '"' + coluna + '"')
        
        #df[coluna][df[coluna] == 'Maioria do tempo + Sempre'] = 1
        #df[coluna][df[coluna] == 'Algumas vezes'] = 0
        #df[coluna][df[coluna] == 'Nunca + Raramente'] = 0
        #print("\n")
    elif 'Nem um nem outro' in df[coluna].unique():
        lista2.append(coluna)
        #print('alteração 2:', '"' + coluna + '"')
        
        #df[coluna][df[coluna] == 'Concordo + Concordo totalmente'] = 0
        #df[coluna][df[coluna] == 'Nem um nem outro'] = 0
        #df[coluna][df[coluna] == 'Discordo + Discordo totalmente'] = 1
        #print("\n")

In [ ]:
lista1

In [ ]:
lista1.remove('28. Os funcionários têm receio de perguntar quando algo não parece certo')
for coluna in lista1:
    df[coluna][df[coluna] == 'Maioria do tempo + Sempre'] = 1
    df[coluna][df[coluna] == 'Algumas vezes'] = 0
    df[coluna][df[coluna] == 'Nunca + Raramente'] = 0

coluna = '28. Os funcionários têm receio de perguntar quando algo não parece certo'
df[coluna][df[coluna] == 'Maioria do tempo + Sempre'] = 0
df[coluna][df[coluna] == 'Algumas vezes'] = 0
df[coluna][df[coluna] == 'Nunca + Raramente'] = 1

print('Ok - Continue')

In [ ]:
lista2

In [ ]:
lista_a2 = ['1. As pessoas se apoiam umas às outras nesta unidade',
            '2. Temos pessoas suficientes para lidar com o volume de trabalho',
            '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
            '4. Nesta unidade, as pessoas se tratam com respeito',
            '6. Estamos ativamente buscando melhorias para a segurança do paciente',
            '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
            '11. Quando uma área nesta unidade fica muito movimentada, as demais ajudam',
            '13. Quando fazemos mudanças para melhorar a segurança do paciente, nós avaliamos sua efetividade',
            '15. A segurança do paciente nunca é sacrificada em prol de se trabalhar mais',
            '18. Nossos procedimentos e sistemas são bons para impedir que os erros aconteçam',
            '19. Meu supervisor/gerente elogia quando vê um trabalho feito de acordo com os procedimentos estabelecidos para a segurança do paciente.',
            '20. Meu supervisor/gerente considera seriamente as sugestões dos funcionários para melhorar a segurança do paciente',
            '32. A administração do hospital cria um ambiente de trabalho que promove a segurança do paciente',
            '35. Existe uma boa cooperação entre as unidades hospitalares que precisam trabalhar juntas',
            '39. As ações da administração do hospital mostram que a segurança do paciente é uma prioridade máxima',
            '41. As unidades do hospital trabalham bem juntas para prestar o melhor atendimento aos pacientes']
for coluna in lista_a2:
    df[coluna][df[coluna] == 'Concordo + Concordo totalmente'] = 1
    df[coluna][df[coluna] == 'Nem um nem outro'] = 0
    df[coluna][df[coluna] == 'Discordo + Discordo totalmente'] = 0
    
print('Ok - Continue')

In [ ]:
lista_a3 = ['5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
            '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
            '8. Os funcionários sentem que seus erros são usados contra eles',
            '10. É apenas por acaso que erros mais sérios não acontecem por aqui',
            '12. Quando um evento é notificado, sentimos que o foco se concentra no indivíduo, e não no problema',
            '14. Trabalhamos “em modo de crise”, tentando fazer coisas demais, rápido demais',
            '16. Os funcionários se preocupam que seus erros sejam registrados em seu arquivo funcional',
            '17. Temos problemas com a segurança do paciente nesta unidade',
            '21. Sempre que a pressão aumenta, meu supervisor/gerente quer que trabalhemos mais rápido, mesmo que isto signifique tomar atalhos',
            '22. Meu supervisor/gerente ignora problemas recorrentes na segurança do paciente',
            '33. As unidades hospitalares não se coordenam bem entre si.',
            '34. Coisas "escapam por entre os dedos” quando os pacientes são transferidos de uma unidade para outra',
            '36. Importantes informações sobre a assistência se perdem durante as mudanças de turno',
            '37. Muitas vezes é desagradável trabalhar unidades com funcionários de outras unidades do hospital',
            '38. Os problemas com frequência ocorrem na troca de informações entre as unidades do hospital',
            '40. A administração do hospital parece se interessar pela segurança do paciente apenas quando acontece um evento adverso',
            '42. As mudanças de turno são problemáticas para os pacientes deste Hospital']

for coluna in lista_a3:
    df[coluna][df[coluna] == 'Concordo + Concordo totalmente'] = 0
    df[coluna][df[coluna] == 'Nem um nem outro'] = 0
    df[coluna][df[coluna] == 'Discordo + Discordo totalmente'] = 1
    
print('Ok - Continue')

In [ ]:
for coluna in df.columns:
    print(coluna, '='*25)
    counts = df[coluna].value_counts()
    percentages = df[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
    res = pd.DataFrame({'counts': counts, 'percentage': percentages})
    print(res)
    print('\n')

In [ ]:
lista = ['1. As pessoas se apoiam umas às outras nesta unidade',
       '2. Temos pessoas suficientes para lidar com o volume de trabalho',
       '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
       '4. Nesta unidade, as pessoas se tratam com respeito',
       '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
       '6. Estamos ativamente buscando melhorias para a segurança do paciente',
       '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
       '8. Os funcionários sentem que seus erros são usados contra eles',
       '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
       '10. É apenas por acaso que erros mais sérios não acontecem por aqui',
       '11. Quando uma área nesta unidade fica muito movimentada, as demais ajudam',
       '12. Quando um evento é notificado, sentimos que o foco se concentra no indivíduo, e não no problema',
       '13. Quando fazemos mudanças para melhorar a segurança do paciente, nós avaliamos sua efetividade',
       '14. Trabalhamos “em modo de crise”, tentando fazer coisas demais, rápido demais',
       '15. A segurança do paciente nunca é sacrificada em prol de se trabalhar mais',
       '16. Os funcionários se preocupam que seus erros sejam registrados em seu arquivo funcional',
       '17. Temos problemas com a segurança do paciente nesta unidade',
       '18. Nossos procedimentos e sistemas são bons para impedir que os erros aconteçam',
       '19. Meu supervisor/gerente elogia quando vê um trabalho feito de acordo com os procedimentos estabelecidos para a segurança do paciente.',
       '20. Meu supervisor/gerente considera seriamente as sugestões dos funcionários para melhorar a segurança do paciente',
       '21. Sempre que a pressão aumenta, meu supervisor/gerente quer que trabalhemos mais rápido, mesmo que isto signifique tomar atalhos',
       '22. Meu supervisor/gerente ignora problemas recorrentes na segurança do paciente',
       '23. Recebemos feedback das mudanças implementadas com base nos eventos notificados',
       '24. Os funcionários falam voluntariamente se vêem algo que possa afetar negativamente o atendimento aos pacientes',
       '25. Somos informados sobre os erros que acontecem nesta unidade',
       '26. Os funcionários sentem-se à vontade para questionar decisões ou ações dos que têm mais autoridade',
       '27. Nesta unidade, discutimos maneiras de impedir que os erros tornem a acontecer',
       '28. Os funcionários têm receio de perguntar quando algo não parece certo',
       '29. Quando acontece um erro, mas ele é identificado e corrigido antes de afetar o paciente, com que frequência é notificado?',
       '30. Quando acontece um erro, mas que não tem potencial de dano ao paciente, com que frequência é notificado?',
       '31. Quando acontece um erro que poderia prejudicar o paciente, mas isto não ocorreu, com que frequência é notificado?',
       'Por favor, dê uma nota geral para a segurança do paciente em sua área/unidade de trabalho hospitalar.',
       '32. A administração do hospital cria um ambiente de trabalho que promove a segurança do paciente',
       '33. As unidades hospitalares não se coordenam bem entre si.',
       '34. Coisas "escapam por entre os dedos” quando os pacientes são transferidos de uma unidade para outra',
       '35. Existe uma boa cooperação entre as unidades hospitalares que precisam trabalhar juntas',
       '36. Importantes informações sobre a assistência se perdem durante as mudanças de turno',
       '37. Muitas vezes é desagradável trabalhar unidades com funcionários de outras unidades do hospital',
       '38. Os problemas com frequência ocorrem na troca de informações entre as unidades do hospital',
       '39. As ações da administração do hospital mostram que a segurança do paciente é uma prioridade máxima',
       '40. A administração do hospital parece se interessar pela segurança do paciente apenas quando acontece um evento adverso',
       '41. As unidades do hospital trabalham bem juntas para prestar o melhor atendimento aos pacientes',
       '42. As mudanças de turno são problemáticas para os pacientes deste Hospital']

# verificando as proporções:

In [ ]:
df_backup1 = df_backup[(df_backup['Ano'] == 2022) & (df_backup['Unidade'] == 'Aliança')].reset_index(drop=True)
df1 = df[(df['Ano'] == 2022) & (df['Unidade'] == 'Anália Franco')].reset_index(drop=True)

for coluna in lista:
    print(coluna, '='*50)
    
    counts = df_backup1[coluna].value_counts()
    percentages = df_backup1[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

    # Usando o método 'agg' para juntar contagens e porcentagens em uma mesma DataFrame
    res = pd.DataFrame({'counts': counts, 'percentage': percentages})
    print(res)
    
    print('\n')
    #
    
    counts = df1[coluna].value_counts()
    percentages = df1[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

    # Usando o método 'agg' para juntar contagens e porcentagens em uma mesma DataFrame
    res = pd.DataFrame({'counts': counts, 'percentage': percentages})
    print(res)
    
    print('\n')

In [ ]:
for i in df.columns:
    print(i, '='*75)
    print(df[i].unique())
    print('\n')

In [ ]:
coluna = 'SEÇÃO G: Número de Eventos Notificados  Nos últimos 12 meses, quantas notificações de evento você preencheu e enviou?'

print(df[coluna].value_counts(), '\n')
df['SEÇÃO G2: Notificou algum evento nos ultimos 12 meses'] = 0
df['SEÇÃO G2: Notificou algum evento nos ultimos 12 meses'][df[coluna] != 'a. Nenhuma notificações de eventos'] = 1
df['SEÇÃO G2: Notificou algum evento nos ultimos 12 meses'][pd.isnull(df[coluna])] = np.nan
print(df['SEÇÃO G2: Notificou algum evento nos ultimos 12 meses'].value_counts(),'\n')

#################################################################################################
df[coluna][df[coluna] == 'a. Nenhuma notificações de eventos'] = '0. Nenhuma notificações de eventos'
df[coluna][df[coluna] == 'b. 1 a 2 notificações de eventos'] = '1. 1 a 2 notificações de eventos'
df[coluna][df[coluna] == 'c. 3 a 5 notificações de eventos'] = '2. 3 a 5 notificações de eventos'
df[coluna][df[coluna] == 'd. 6 a 10 notificações de eventos'] = '3. 6 a 10 notificações de eventos'
df[coluna][df[coluna] == 'e. 11 a 20 notificações de eventos'] = '4. 11 a 20 notificações de eventos'
df[coluna][df[coluna] == 'f. 21 notificações de eventos ou mais'] = '5. 21 notificações de eventos ou mais'
print(df[coluna].value_counts())

# Gerando as variáveis dummy
dummies = pd.get_dummies(df[coluna], prefix=coluna)
df = pd.concat([df, dummies], axis=1)

In [ ]:
coluna = 'SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'
df[coluna].unique()

In [ ]:
#Na unidade tem muito funcionario que sobrecarregado ?
coluna = 'SEÇÃO H: 3.Tipicamente, quantas horas por semana você trabalha neste hospital?  '
print(df[coluna].value_counts(),'\n')

df['SEÇÃO H: 3.2 Trabalha mais que 40horas semanais'] = 1
df['SEÇÃO H: 3.2 Trabalha mais que 40horas semanais'][(df[coluna] == 'a. Menos de 20 horas por semana') |
                                                        (df[coluna] == 'b. 20 a 39 horas por semana')] = 0
df['SEÇÃO H: 3.2 Trabalha mais que 40horas semanais'][pd.isnull(df[coluna])] = np.nan
print(df['SEÇÃO H: 3.2 Trabalha mais que 40horas semanais'].value_counts(), '\n')

##########################################################################
df['SEÇÃO H: 3.2 Trabalha mais que 60horas semanais'] = 1
df['SEÇÃO H: 3.2 Trabalha mais que 60horas semanais'][(df[coluna] == 'a. Menos de 20 horas por semana') |
                                                      (df[coluna] == 'b. 20 a 39 horas por semana') |
                                                      (df[coluna] == 'c. 40 a 59 horas por semana')] = 0
df['SEÇÃO H: 3.2 Trabalha mais que 60horas semanais'][pd.isnull(df[coluna])] = np.nan
print(df['SEÇÃO H: 3.2 Trabalha mais que 60horas semanais'].value_counts(), '\n')

##########################################################################
df['SEÇÃO H: 3.2 Trabalha mais que 80horas semanais'] = 0
df['SEÇÃO H: 3.2 Trabalha mais que 80horas semanais'][(df[coluna] == 'e. 80 a 99 horas por semana') |
                                                      (df[coluna] == 'f. 100 horas por semana ou mais')] = 1
df['SEÇÃO H: 3.2 Trabalha mais que 80horas semanais'][pd.isnull(df[coluna])] = np.nan
print(df['SEÇÃO H: 3.2 Trabalha mais que 80horas semanais'].value_counts(), '\n')

##########################################################################
coluna2 = 'SEÇÃO H: Horas trabalhadas semanais'
df[coluna2] = np.nan
df[coluna2][df[coluna] == 'a. Menos de 20 horas por semana'] = '0. 20 a 39'
df[coluna2][df[coluna] == 'b. 20 a 39 horas por semana'] = '0. 20 a 39'
df[coluna2][df[coluna] == 'c. 40 a 59 horas por semana'] = '1. 40 a - 79'
df[coluna2][df[coluna] == 'd. 60 a 79 horas por semana'] = '1. 40 a - 79'
df[coluna2][df[coluna] == 'e. 80 a 99 horas por semana'] = '2. > 80'
df[coluna2][df[coluna] == 'f. 100 horas por semana ou mais'] = '2. > 80'

########################################################################## ARRUMANDO OS NIVEIS
coluna = "SEÇÃO H: 3.Tipicamente, quantas horas por semana você trabalha neste hospital?  "
df[coluna][df[coluna] == 'a. Menos de 20 horas por semana'] = '1. Menos de 20 horas por semana'
df[coluna][df[coluna] == 'b. 20 a 39 horas por semana'] = '2. 20 a 39 horas por semana'
df[coluna][df[coluna] == 'c. 40 a 59 horas por semana'] = '3. 40 a 59 horas por semana'
df[coluna][df[coluna] == 'd. 60 a 79 horas por semana'] = '4. 60 a 79 horas por semana'
df[coluna][df[coluna] == 'e. 80 a 99 horas por semana'] = '5. 80 a 99 horas por semana'
df[coluna][df[coluna] == 'f. 100 horas por semana ou mais'] = '6. 100 horas por semana ou mais'

In [ ]:
coluna = 'Por favor, dê uma nota geral para a segurança do paciente em sua área/unidade de trabalho hospitalar.'
print(df[coluna].value_counts(),'\n')

# Gerando as variáveis dummy
dummies = pd.get_dummies(df[coluna], prefix=coluna)
df = pd.concat([df, dummies], axis=1)

df['Nota Geral == Satisfatoria ?'] = 1
df['Nota Geral == Satisfatoria ?'][(df[coluna] == 'Ruim') |
                                   (df[coluna] == 'Muito ruim')] = 0
df['Nota Geral == Satisfatoria ?'][pd.isnull(df[coluna])] = np.nan
print(df['Nota Geral == Satisfatoria ?'].value_counts(), '\n')

##########################################################################
df['Nota Geral == Boa ?'] = 1
df['Nota Geral == Boa ?'][(df[coluna] == 'Ruim') |
                          (df[coluna] == 'Muito ruim') |
                         (df[coluna] == 'Satisfatória')] = 0
df['Nota Geral == Boa ?'][pd.isnull(df[coluna])] = np.nan
print(df['Nota Geral == Boa ?'].value_counts(), '\n')

##########################################################################
df['Nota Geral'] = 1
df['Nota Geral'][(df[coluna] == 'Ruim') |
                   (df[coluna] == 'Muito ruim')] = 0
df['Nota Geral'][df[coluna] == 'Satisfatória'] = 1
df['Nota Geral'][(df[coluna] == 'Excelente') |
                   (df[coluna] == 'Muito boa')] = 2
df['Nota Geral'][pd.isnull(df[coluna])] = np.nan
print(df['Nota Geral'].value_counts(), '\n')

########################################################################## ARRUMANDO NIVEIS
df[coluna][df[coluna] == 'Muito ruim'] = '0. Muito ruim'
df[coluna][df[coluna] == 'Ruim'] = '1. Ruim'
df[coluna][df[coluna] == 'Satisfatória'] = '2. Satisfatória'
df[coluna][df[coluna] == 'Excelente'] = '3. Excelente'
df[coluna][df[coluna] == 'Muito boa'] = '4. Muito boa'
print(df[coluna].value_counts(),'\n')

In [ ]:
df.columns

In [ ]:
print(df['SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'].value_counts(), '\n')

#Verificar a rotatividade do hospital
df['SEÇÃO H: 1.2 trabalha a mais de 1 ano no hospital?'] = 0
df['SEÇÃO H: 1.2 trabalha a mais de 1 ano no hospital?'][df['SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'] != 'menos de 1 ano'] = 1
df['SEÇÃO H: 1.2 trabalha a mais de 1 ano no hospital?'][pd.isnull(df['SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'])] = np.nan
print(df['SEÇÃO H: 1.2 trabalha a mais de 1 ano no hospital?'].value_counts(), '\n')

df['SEÇÃO H: 1.3 trabalha a mais de 5 anos no hospital?'] = 1
df['SEÇÃO H: 1.3 trabalha a mais de 5 anos no hospital?'][(df['SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'] == 'menos de 1 ano') | 
                                                          (df['SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'] == '1 a 5 anos')] = 0
df['SEÇÃO H: 1.3 trabalha a mais de 5 anos no hospital?'][pd.isnull(df['SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'])] = np.nan
print(df['SEÇÃO H: 1.3 trabalha a mais de 5 anos no hospital?'].value_counts())


In [ ]:
colunas = ["SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?",
           'SEÇÃO H: 2.Há quanto tempo você trabalha nesta área/unidade de trabalho hospitalar?',
           "6. Há quanto tempo você trabalha em sua profissão ou especialização atual?  "]

for coluna in colunas:
    print('\n')
    df[coluna][df[coluna] == 'a. Menos de 1 ano'] = '0. Menos de 1 ano'
    df[coluna][df[coluna] == 'Menos de 1 ano'] = '0. Menos de 1 ano'
    df[coluna][df[coluna] == 'menos de 1 ano'] = '0. Menos de 1 ano'
    
    df[coluna][df[coluna] == 'b. 1 a 5 anos'] = '1. 1 a 5 anos'
    df[coluna][df[coluna] == '1 a 5 anos'] = '1. 1 a 5 anos'
    
    df[coluna][df[coluna] == 'c. 6 a 10 anos'] = '2. 6 a 10 anos'
    df[coluna][df[coluna] == '6 a 10 anos'] = '2. 6 a 10 anos'
    
    df[coluna][df[coluna] == 'd. 11 a 15 anos'] = '3. 11 a 15 anos'
    df[coluna][df[coluna] == '11 a 15 anos'] = '3. 11 a 15 anos'
    
    df[coluna][df[coluna] == 'e. 16 a 20 anos'] = '4. 16 a 20 anos'
    df[coluna][df[coluna] == '16 a 20 anos'] = '4. 16 a 20 anos'
    
    df[coluna][df[coluna] == 'f. 21 anos ou mais'] = '5. 21 anos ou mais'
    df[coluna][df[coluna] == '21 anos ou mais'] = '5. 21 anos ou mais'
    
    print(df[coluna].value_counts())

In [ ]:
coluna = 'SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'
# Gerando as variáveis dummy
dummies = pd.get_dummies(df[coluna], prefix=coluna)
df = pd.concat([df, dummies], axis=1)

In [ ]:
df.columns

In [ ]:
df.columns = [col.strip() for col in df.columns]

coluna = "6. Há quanto tempo você trabalha em sua profissão ou especialização atual?"
# Gerando as variáveis dummy
dummies = pd.get_dummies(df[coluna], prefix=coluna)
df = pd.concat([df, dummies], axis=1)

In [ ]:
pasta = 'C:/Users/gabri/OneDrive/Documentos/Anestesia - CMA/Cultura de Segurança/2 - Tabelas com Escores Anuais'
df.to_excel(pasta + '/' + '0 - Dados_Limpos.xlsx', index=False)
df_bruto = df.copy()
len(df)

# FILTRO FINAL

In [ ]:
df_filter = df[df['Ano'] == 2022]
df_filter = df_filter.dropna()
df_filter = df_filter.reset_index(drop=True)
df_filter.to_excel(pasta + '/' + '0 - Dados_Limpos_estudo_de_coorte.xlsx', index=False)
len(df_filter)

In [ ]:
stop

# TESTES / ANALISES =================================

In [ ]:
for i in lista:
    print('coluna:', i)
    print('0: mean(', round(resultado[i][resultado['acreditacao'] == 0].mean(),4), ') ; median(', round(resultado[i][resultado['acreditacao'] == 0].median(),4), ')')
    print('1: mean(', round(resultado[i][resultado['acreditacao'] == 1].mean(),4), ') ; median(', round(resultado[i][resultado['acreditacao'] == 1].median(),4), ')')
    
    a = resultado[i][resultado['acreditacao'] == 0] # amostra a
    b = resultado[i][resultado['acreditacao'] == 1] # amostra b
    ttest = round(stats.ttest_ind(a,b)[1], 4)
    mann = round(stats.mannwhitneyu(a, b)[1], 4)
    print('pvalor:  '+str(ttest)+'             '+str(mann))
    
    grupos = resultado['acreditacao'].unique()

    plt.figure(figsize=(10,2))

    for grupo in grupos:

        if grupo == 0:
            label_grupo = "Sem Acreditação"
        else:
            label_grupo = "Com Acreditação"

        sns.kdeplot(resultado[resultado['acreditacao'] == grupo][i], label=label_grupo)

    plt.axvline(x = resultado[i][resultado['acreditacao'] == 0].mean(), color='red', linestyle='--', label='Média sem')
    plt.axvline(x = resultado[i][resultado['acreditacao'] == 0].median(), color='blue', linestyle='--', label='Mediana sem')

    plt.axvline(x = resultado[i][resultado['acreditacao'] == 1].mean(), color='orange', linestyle='--', label='Média com')
    plt.axvline(x = resultado[i][resultado['acreditacao'] == 1].median(), color='brown', linestyle='--', label='Mediana com')

    plt.title(i)
    plt.legend()
    plt.show()
    
    print('='*125)

# APLICANDO ANOVA:

In [ ]:
resultado1 = resultado.rename(columns={"42. As mudanças de turno são problemáticas para os pacientes deste Hospital": "var_num"})

# Ajuste o modelo
modelo = ols('var_num ~ C(acreditacao) + C(Regional) + C(acreditacao):C(Regional)', data=resultado1).fit()

# Realize a ANOVA e imprima a tabela de resultados
anova_table = sm.stats.anova_lm(modelo, typ=2)
print(anova_table)


In [ ]:
modelo = ols('var_num ~ C(acreditacao)', data = resultado1).fit()

# Realize a ANOVA e imprima a tabela de resultados
anova_table = sm.stats.anova_lm(modelo, typ=2)
print(anova_table)

In [ ]:
resultado['Regional2'] = np.nan
resultado['Regional2'][(resultado['Regional'] == 'SUL') | (resultado['Regional'] == 'SE')] = 'SUL'
resultado['Regional2'][(resultado['Regional'] == 'N') | (resultado['Regional'] == 'NE')] = 'NORTE'
resultado['Regional2'][resultado['Regional'] == 'CO'] = 'CO'
print(resultado['Regional2'].value_counts())

##########
resultado['Regional3'] = resultado['Regional2']
resultado['Regional3'][(resultado['Regional2'] == 'CO') | (resultado['Regional2'] == 'NORTE')] = 'NORTE_OC'
print(resultado['Regional3'].value_counts())

In [ ]:
#resultado.to_excel('tabela3.xlsx')

In [ ]:
for i in lista:
    print('coluna:', i)
    print('NORTE_OC: mean(', round(resultado[i][resultado['Regional3'] == 'NORTE_OC'].mean(),4), ') ; median(', round(resultado[i][resultado['Regional3'] == 'NORTE_OC'].median(),4), ')')
    print('SUL: mean(', round(resultado[i][resultado['Regional3'] == 'SUL'].mean(),4), ') ; median(', round(resultado[i][resultado['Regional3'] == 'SUL'].median(),4), ')')
    
    a = resultado[i][resultado['Regional3'] == 'NORTE_OC'] # amostra a
    b = resultado[i][resultado['Regional3'] == 'SUL'] # amostra b
    ttest = round(stats.ttest_ind(a,b)[1], 4)
    mann = round(stats.mannwhitneyu(a, b)[1], 4)
    print('pvalor:  '+str(ttest)+'             '+str(mann))
    
    grupos = resultado['Regional3'].unique()

    plt.figure(figsize=(10,2))

    for grupo in grupos:

        if grupo == 'NORTE_OC':
            label_grupo = "NORTE"
        else:
            label_grupo = "SUL"

        sns.kdeplot(resultado[resultado['Regional3'] == grupo][i], label=label_grupo)

    plt.axvline(x = resultado[i][resultado['Regional3'] == 'NORTE_OC'].mean(), color='red', linestyle='--', label='Média NORTE')
    plt.axvline(x = resultado[i][resultado['Regional3'] == 'NORTE_OC'].median(), color='blue', linestyle='--', label='Mediana NORTE')

    plt.axvline(x = resultado[i][resultado['Regional3'] == 'SUL'].mean(), color='orange', linestyle='--', label='Média SUL')
    plt.axvline(x = resultado[i][resultado['Regional3'] == 'SUL'].median(), color='brown', linestyle='--', label='Mediana SUL')

    plt.title(i)
    plt.legend()
    plt.show()
    
    print('='*125)

In [ ]:
df

In [ ]:
df = pd.read_excel("tabela3.xlsx")
print(len(df))
df.sample(5)

# VERFICANDO SE HOUVE MELHORA

In [ ]:
for i in lista:
    print('coluna:', i)
    print('0: mean(', round(tabela[i][tabela['acreditacao'] == 0].mean(),4), ') ; median(', round(tabela[i][tabela['acreditacao'] == 0].median(),4), ')')
    print('1: mean(', round(tabela[i][tabela['acreditacao'] == 1].mean(),4), ') ; median(', round(tabela[i][tabela['acreditacao'] == 1].median(),4), ')')
    
    a = tabela[i][tabela['acreditacao'] == 0] # amostra a
    b = tabela[i][tabela['acreditacao'] == 1] # amostra b
    ttest = round(stats.ttest_ind(a,b)[1], 4)
    mann = round(stats.mannwhitneyu(a, b)[1], 4)
    print('pvalor:  '+str(ttest)+'             '+str(mann))
    
    grupos = tabela['acreditacao'].unique()

    plt.figure(figsize=(10,2))

    for grupo in grupos:

        if grupo == 0:
            label_grupo = "Sem Acreditação"
        else:
            label_grupo = "Com Acreditação"

        sns.kdeplot(tabela[tabela['acreditacao'] == grupo][i], label=label_grupo)

    plt.axvline(x = tabela[i][tabela['acreditacao'] == 0].mean(), color='red', linestyle='--', label='Média sem')
    plt.axvline(x = tabela[i][tabela['acreditacao'] == 0].median(), color='blue', linestyle='--', label='Mediana sem')

    plt.axvline(x = tabela[i][tabela['acreditacao'] == 1].mean(), color='orange', linestyle='--', label='Média com')
    plt.axvline(x = tabela[i][tabela['acreditacao'] == 1].median(), color='brown', linestyle='--', label='Mediana com')

    plt.title(i)
    plt.legend()
    plt.show()
    
    print('='*125)

In [ ]:
count = 0
for unidade in sorted(df['Unidade'].unique()):
    if len(df['acreditacao'][df['Unidade'] == unidade].unique()) == 1:
        if df['acreditacao'][df['Unidade'] == unidade].unique()[0] == 0:
            print(unidade, '='*25)
            print(df[df['Unidade'] == unidade].groupby(['Ano', 'acreditacao']).size(), '\n')
            count += 1
            
print('='*25, 'QUANTIDADE:', count, '='*25)

In [ ]:
count = 0 
for unidade in sorted(df['Unidade'].unique()):
    if len(df['acreditacao'][df['Unidade'] == unidade].unique()) == 1:
        if df['acreditacao'][df['Unidade'] == unidade].unique()[0] == 1:
            print(unidade, '='*25)
            print(df[df['Unidade'] == unidade].groupby(['Ano', 'acreditacao']).size(), '\n')
            count += 1
            
print('='*25, 'QUANTIDADE:', count, '='*25)

# Existe uma mudança no score com o passar do tempo

In [ ]:
#tabela = df[df['acreditacao'] == 1].reset_index(drop=True)
#tabela = df
count = 0 
for coluna in lista:
    subtitulo = ''
    mediana = df.groupby(['Ano', 'acreditacao'])[coluna].median().unstack().reset_index()
    
    ##############################################################################################
    for acre in range(0, 2):
        X = df['Ano'][df['acreditacao'] == acre]
        Y = df[coluna][df['acreditacao'] == acre]
        X = sm.add_constant(X)
        model = sm.OLS(Y, X)
        results = model.fit()
        p_valor = round(results.pvalues['Ano'], 3)
        if p_valor < 0.001:p_valor="<0.001"
        coef = str(round(results.params['Ano'], 2))
        interval = results.conf_int(alpha=0.05).loc['Ano']
        interval_0 = str(round(interval[0], 2))
        interval_1 = str(round(interval[1], 2))
        subtitulo = coef + ' (' + interval_0 + ' - ' + interval_1 + ')'
        subtitulo = 'p-valor = ' + str(p_valor) + ' | '+ subtitulo
        subtitulo = subtitulo + " | R² = " + str(round(results.rsquared, 2))
        
        subtitulo = str(round(Y.mean(), 2)) + " | " + subtitulo

        plt.figure(figsize=(10,3))
        if acre == 0:
            cor = "red"
            plt.title(coluna, fontproperties=fonte, fontsize=16)
        else:
            cor = 'blue'
        sns.scatterplot(x="Ano", y=coluna, data=df[df['acreditacao'] == acre],
                           edgecolor = "black", color = cor)
        sns.lineplot(x='Ano', y=acre, data = mediana, color = 'green')
        sns.lineplot(x="Ano", y=coluna, data=df[df['acreditacao'] == acre], color = 'red')

        plt.xlabel(subtitulo, fontproperties=fonte, fontsize=13)
        plt.ylabel(str(acre)+'acreditação', fontproperties=fonte, fontsize=13)
        plt.ylim(0, 100)
        plt.show()

        ##############################################################################################

In [ ]:
#tabela = df[df['acreditacao'] == 1].reset_index(drop=True)
#tabela = df
count = 0 
for coluna in lista:
    subtituloo = ''
    plt.figure(figsize=(10,3))
    sns.lineplot(x="Ano", y=coluna, hue='acreditacao', data=df)
    plt.title(coluna, fontproperties=fonte, fontsize=16)
    plt.ylabel('Score Médio', fontproperties=fonte, fontsize=13)
    #plt.ylim(0, 100)
    
    ##############################################################################################
    for acre in range(0, 2):
        X = df['Ano'][df['acreditacao'] == acre]
        Y = df[coluna][df['acreditacao'] == acre]
        X = sm.add_constant(X)
        model = sm.OLS(Y, X)
        results = model.fit()
        p_valor = round(results.pvalues['Ano'], 3)
        if p_valor < 0.001:p_valor="<0.001"
        coef = str(round(results.params['Ano'], 2))
        interval = results.conf_int(alpha=0.05).loc['Ano']
        interval_0 = str(round(interval[0], 2))
        interval_1 = str(round(interval[1], 2))
        subtitulo = coef + ' (' + interval_0 + ' - ' + interval_1 + ')'
        subtitulo = 'p-valor = ' + str(p_valor) + ' | '+ subtitulo
        subtitulo = subtitulo + " | R² = " + str(round(results.rsquared, 2))
        
        subtitulo = str(round(Y.mean(), 2)) + " | " + subtitulo
        subtitulo = str(acre) + "=> " + subtitulo
        subtitulo = subtitulo + " | " + str(round(durbin_watson(results.resid), 2))
        
        if acre == 1:
            subtituloo = subtituloo + '\n' + subtitulo
        else:
            subtituloo = subtituloo + subtitulo
    
    plt.xlabel(subtituloo, fontproperties=fonte, fontsize=13)
    plt.show()

        ##############################################################################################

In [ ]:
durbinWatson = durbin_watson(results.resid)
round(durbinWatson, 2)

In [ ]:


for i in lista:
    print('='*25, i, '='*25)
    Y = df[i]
    X = df[['Ano', 'acreditacao']]
    X = sm.add_constant(X)
    model = sm.OLS(Y, X)
    results = model.fit()
    print(results.summary())
    durbinWatson = durbin_watson(results.resid)
    print('\n')
    print('O valor de Durbin-Watson é:', durbinWatson)
    print('\n')

In [ ]:
df

In [ ]:
df

In [ ]:
lista2 = ['Abertura de comunicação','Feedback e comunicação sobre erros',
          'Frequência de eventos relatados','Apoio de gestão hospitalar para segurança do paciente',
          'Trabalho em equipe entre as unidades hospitalares','Trabalho em equipe no âmbito das unidades',
          'Transferências internas e passagens de plantão','Aprendizado organizacional - melhoria contínua',
          'Expectativas de promoção da segurança dos supervisores e gerentes','Percepção gerais sobre segurança',
          'Respostas não punitivas aos erros','Staffing']
count = 0 
for coluna in lista2:
    subtituloo = ''
    plt.figure(figsize=(10,3))
    sns.lineplot(x="Ano", y=coluna, hue='acreditacao', data=df)
    plt.title(coluna, fontproperties=fonte, fontsize=16)
    plt.ylabel('Score Médio', fontproperties=fonte, fontsize=13)
    #plt.ylim(0, 100)
    
    ##############################################################################################
    for acre in range(0, 2):
        X = df['Ano'][df['acreditacao'] == acre]
        Y = df[coluna][df['acreditacao'] == acre]
        X = sm.add_constant(X)
        model = sm.OLS(Y, X)
        results = model.fit()
        p_valor = round(results.pvalues['Ano'], 3)
        if p_valor < 0.001:p_valor="<0.001"
        coef = str(round(results.params['Ano'], 2))
        interval = results.conf_int(alpha=0.05).loc['Ano']
        interval_0 = str(round(interval[0], 2))
        interval_1 = str(round(interval[1], 2))
        subtitulo = coef + ' (' + interval_0 + ' - ' + interval_1 + ')'
        subtitulo = 'p-valor = ' + str(p_valor) + ' | '+ subtitulo
        subtitulo = subtitulo + " | R² = " + str(round(results.rsquared, 2))
        
        subtitulo = str(round(Y.mean(), 2)) + " | " + subtitulo
        subtitulo = str(acre) + "=> " + subtitulo
        subtitulo = subtitulo + " | " + str(round(durbin_watson(results.resid), 2))
        
        if acre == 1:
            subtituloo = subtituloo + '\n' + subtitulo
        else:
            subtituloo = subtituloo + subtitulo
    
    plt.xlabel(subtituloo, fontproperties=fonte, fontsize=13)
    plt.show()

        ##############################################################################################

In [ ]:
tabela

In [ ]:
for i in lista2:
    print('coluna:', i)
    print('0: mean(', round(tabela[i][tabela['acreditacao'] == 0].mean(),4), ') ; median(', round(tabela[i][tabela['acreditacao'] == 0].median(),4), ')')
    print('1: mean(', round(tabela[i][tabela['acreditacao'] == 1].mean(),4), ') ; median(', round(tabela[i][tabela['acreditacao'] == 1].median(),4), ')')
    
    a = tabela[i][tabela['acreditacao'] == 0] # amostra a
    b = tabela[i][tabela['acreditacao'] == 1] # amostra b
    ttest = round(stats.ttest_ind(a,b)[1], 4)
    mann = round(stats.mannwhitneyu(a, b)[1], 4)
    print('pvalor:  '+str(ttest)+'             '+str(mann))
    
    grupos = tabela['acreditacao'].unique()

    plt.figure(figsize=(10,2))

    for grupo in grupos:

        if grupo == 0:
            label_grupo = "Sem Acreditação"
        else:
            label_grupo = "Com Acreditação"

        sns.kdeplot(tabela[tabela['acreditacao'] == grupo][i], label=label_grupo)

    plt.axvline(x = tabela[i][tabela['acreditacao'] == 0].mean(), color='red', linestyle='--', label='Média sem')
    plt.axvline(x = tabela[i][tabela['acreditacao'] == 0].median(), color='blue', linestyle='--', label='Mediana sem')

    plt.axvline(x = tabela[i][tabela['acreditacao'] == 1].mean(), color='orange', linestyle='--', label='Média com')
    plt.axvline(x = tabela[i][tabela['acreditacao'] == 1].median(), color='brown', linestyle='--', label='Mediana com')

    plt.title(i)
    plt.legend()
    plt.show()
    
    print('='*125)